In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
data=pd.read_csv(r"C:\Users\divyam.arora\Desktop\New Folder 3\machine-learning-specialization-master\course-3\datasets\amazon_baby_subset.csv",sep=',')
import string
data['review'].fillna(value='',inplace=True)
data['review']=data['review'].astype(str)
def remove_punctuation(text):
    return text.translate(str.maketrans('','',string.punctuation)) 

data['review_clean']=data['review'].apply(remove_punctuation)
important_words=["baby", "one", "great", "love", "use", "would", "like", "easy", "little", "seat", "old", "well", "get", "also", "really", "son", "time", "bought", "product", "good", "daughter", "much", "loves", "stroller", "put", "months", "car", "still", "back", "used", "recommend", "first", "even", "perfect", "nice", "bag", "two", "using", "got", "fit", "around", "diaper", "enough", "month", "price", "go", "could", "soft", "since", "buy", "room", "works", "made", "child", "keep", "size", "small", "need", "year", "big", "make", "take", "easily", "think", "crib", "clean", "way", "quality", "thing", "better", "without", "set", "new", "every", "cute", "best", "bottles", "work", "purchased", "right", "lot", "side", "happy", "comfortable", "toy", "able", "kids", "bit", "night", "long", "fits", "see", "us", "another", "play", "day", "money", "monitor", "tried", "thought", "never", "item", "hard", "plastic", "however", "disappointed", "reviews", "something", "going", "pump", "bottle", "cup", "waste", "return", "amazon", "different", "top", "want", "problem", "know", "water", "try", "received", "sure", "times", "chair", "find", "hold", "gate", "open", "bottom", "away", "actually", "cheap", "worked", "getting", "ordered", "came", "milk", "bad", "part", "worth", "found", "cover", "many", "design", "looking", "weeks", "say", "wanted", "look", "place", "purchase", "looks", "second", "piece", "box", "pretty", "trying", "difficult", "together", "though", "give", "started", "anything", "last", "company", "come", "returned", "maybe", "took", "broke", "makes", "stay", "instead", "idea", "head", "said", "less", "went", "working", "high", "unit", "seems", "picture", "completely", "wish", "buying", "babies", "won", "tub", "almost", "either"]
for word in important_words:
    data[word] = data['review_clean'].apply(lambda s : s.split().count(word))
features=data.columns.tolist()
features.remove('sentiment')
features.remove('name')
features.remove('review')
features.remove('rating')
features.remove('review_clean')
label=data['sentiment']
features=['constant'] + features
data['constant'] = 1
train_data, validation_data,train_label,validation_label=train_test_split(data[features].as_matrix(),label.as_matrix(),test_size=0.2,random_state=3)


In [2]:
#link function for logistic regression
def predict_probability(feature_matrix, coefficients):
    # Take dot product of feature_matrix and coefficients  
    # YOUR CODE HERE
    score=np.dot(feature_matrix,coefficients)
    
    # Compute P(y_i = +1 | x_i, w) using the link function
    # YOUR CODE HERE
    predictions = 1/(1+np.exp(-score))
    
    # return predictions
    return predictions

In [3]:
def feature_derivative(errors, feature):     
    # Compute the dot product of errors and feature
    value=np.dot(errors,feature)
    derivative = np.sum(value)
    
    # Return the derivative
    return derivative
def feature_derivative_with_L2(errors, feature, coefficient, l2_penalty, feature_is_constant): 
    
    # Compute the dot product of errors and feature
    value=np.dot(errors,feature)
    derivative = np.sum(value)

    # add L2 penalty term for any feature that isn't the intercept.
    if not feature_is_constant: 
        ## YOUR CODE HERE
        derivative=derivative-(2*(l2_penalty*coefficient))
        
    return derivative

In [4]:
def compute_log_likelihood_with_L2(feature_matrix, sentiment, coefficients, l2_penalty):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    
    lp = np.sum((indicator-1)*scores - np.log(1. + np.exp(-scores))) - l2_penalty*np.sum(coefficients[1:]**2)
    
    return lp

In [5]:
#logistic regression
from math import sqrt

def logistic_regression_with_L2(feature_matrix, sentiment, initial_coefficients, step_size, l2_penalty, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in range(max_iter):
        # Predict P(y_i = +1|x_i,w) using your predict_probability() function
        ## YOUR CODE HERE
        predictions = predict_probability(feature_matrix, coefficients)
        
        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)
        
        # Compute the errors as indicator - predictions
        errors = indicator - predictions
        for j in range(len(coefficients)): # loop over each coefficient
            is_intercept = (j == 0)
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j].
            # Compute the derivative for coefficients[j]. Save it in a variable called derivative
            ## YOUR CODE HERE
            derivative = feature_derivative(errors, feature_matrix[:,j])
            
            # add the step size times the derivative to the current coefficient
            ## YOUR CODE HERE
            coefficients[j]=coefficients[j]+(derivative*step_size)
        
        # Checking whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood_with_L2(feature_matrix, sentiment, coefficients, l2_penalty)
            print('iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp))
    return coefficients

In [8]:
# run with L2 = 0
coefficients_0_penalty = logistic_regression_with_L2(train_data, train_label,
                                                     initial_coefficients=np.zeros(194),
                                                     step_size=5e-6, l2_penalty=0, max_iter=501)

iteration   0: log likelihood of observed labels = -29244.41002597
iteration   1: log likelihood of observed labels = -29067.83207373
iteration   2: log likelihood of observed labels = -28898.25174925
iteration   3: log likelihood of observed labels = -28734.98531155
iteration   4: log likelihood of observed labels = -28577.52232358
iteration   5: log likelihood of observed labels = -28425.46142638
iteration   6: log likelihood of observed labels = -28278.47182690
iteration   7: log likelihood of observed labels = -28136.27006088
iteration   8: log likelihood of observed labels = -27998.60582965
iteration   9: log likelihood of observed labels = -27865.25324782
iteration  10: log likelihood of observed labels = -27736.00533299
iteration  11: log likelihood of observed labels = -27610.67044681
iteration  12: log likelihood of observed labels = -27489.06991675
iteration  13: log likelihood of observed labels = -27371.03637748
iteration  14: log likelihood of observed labels = -27256.4125

In [9]:
# run with L2 = 4
coefficients_4_penalty = logistic_regression_with_L2(train_data, train_label,
                                                      initial_coefficients=np.zeros(194),
                                                      step_size=5e-6, l2_penalty=4, max_iter=501)

iteration   0: log likelihood of observed labels = -29244.41375958
iteration   1: log likelihood of observed labels = -29067.84667170
iteration   2: log likelihood of observed labels = -28898.28389643
iteration   3: log likelihood of observed labels = -28735.04130516
iteration   4: log likelihood of observed labels = -28577.60811457
iteration   5: log likelihood of observed labels = -28425.58265020
iteration   6: log likelihood of observed labels = -28278.63382776
iteration   7: log likelihood of observed labels = -28136.47791168
iteration   8: log likelihood of observed labels = -27998.86434911
iteration   9: log likelihood of observed labels = -27865.56701570
iteration  10: log likelihood of observed labels = -27736.37870381
iteration  11: log likelihood of observed labels = -27611.10756253
iteration  12: log likelihood of observed labels = -27489.57471850
iteration  13: log likelihood of observed labels = -27371.61261652
iteration  14: log likelihood of observed labels = -27257.0638

In [10]:
# run with L2 = 10
coefficients_10_penalty = logistic_regression_with_L2(train_data, train_label,
                                                      initial_coefficients=np.zeros(194),
                                                      step_size=5e-6, l2_penalty=10, max_iter=501)

iteration   0: log likelihood of observed labels = -29244.41936000
iteration   1: log likelihood of observed labels = -29067.86856866
iteration   2: log likelihood of observed labels = -28898.33211720
iteration   3: log likelihood of observed labels = -28735.12529558
iteration   4: log likelihood of observed labels = -28577.73680104
iteration   5: log likelihood of observed labels = -28425.76448594
iteration   6: log likelihood of observed labels = -28278.87682905
iteration   7: log likelihood of observed labels = -28136.78968788
iteration   8: log likelihood of observed labels = -27999.25212831
iteration   9: log likelihood of observed labels = -27866.03766752
iteration  10: log likelihood of observed labels = -27736.93876004
iteration  11: log likelihood of observed labels = -27611.76323610
iteration  12: log likelihood of observed labels = -27490.33192111
iteration  13: log likelihood of observed labels = -27372.47697508
iteration  14: log likelihood of observed labels = -27258.0406

In [11]:
# run with L2 = 1e2
coefficients_1e2_penalty = logistic_regression_with_L2(train_data, train_label,
                                                       initial_coefficients=np.zeros(194),
                                                       step_size=5e-6, l2_penalty=1e2, max_iter=501)

iteration   0: log likelihood of observed labels = -29244.50336630
iteration   1: log likelihood of observed labels = -29068.19702307
iteration   2: log likelihood of observed labels = -28899.05542867
iteration   3: log likelihood of observed labels = -28736.38515182
iteration   4: log likelihood of observed labels = -28579.66709818
iteration   5: log likelihood of observed labels = -28428.49202194
iteration   6: log likelihood of observed labels = -28282.52184844
iteration   7: log likelihood of observed labels = -28141.46633081
iteration   8: log likelihood of observed labels = -28005.06881623
iteration   9: log likelihood of observed labels = -27873.09744480
iteration  10: log likelihood of observed labels = -27745.33960351
iteration  11: log likelihood of observed labels = -27621.59833974
iteration  12: log likelihood of observed labels = -27501.68996032
iteration  13: log likelihood of observed labels = -27385.44235349
iteration  14: log likelihood of observed labels = -27272.6937

In [12]:
# run with L2 = 1e3
coefficients_1e3_penalty = logistic_regression_with_L2(train_data, train_label,
                                                       initial_coefficients=np.zeros(194),
                                                       step_size=5e-6, l2_penalty=1e3, max_iter=501)

iteration   0: log likelihood of observed labels = -29245.34342928
iteration   1: log likelihood of observed labels = -29071.48156711
iteration   2: log likelihood of observed labels = -28906.28854345
iteration   3: log likelihood of observed labels = -28748.98371419
iteration   4: log likelihood of observed labels = -28598.97006955
iteration   5: log likelihood of observed labels = -28455.76738196
iteration   6: log likelihood of observed labels = -28318.97204229
iteration   7: log likelihood of observed labels = -28188.23276018
iteration   8: log likelihood of observed labels = -28063.23569550
iteration   9: log likelihood of observed labels = -27943.69521761
iteration  10: log likelihood of observed labels = -27829.34803820
iteration  11: log likelihood of observed labels = -27719.94937608
iteration  12: log likelihood of observed labels = -27615.27035242
iteration  13: log likelihood of observed labels = -27515.09613751
iteration  14: log likelihood of observed labels = -27419.2245

In [13]:
# run with L2 = 1e5
coefficients_1e5_penalty = logistic_regression_with_L2(train_data, train_label,
                                                       initial_coefficients=np.zeros(194),
                                                       step_size=5e-6, l2_penalty=1e5, max_iter=501)

iteration   0: log likelihood of observed labels = -29337.75035722
iteration   1: log likelihood of observed labels = -29432.78141131
iteration   2: log likelihood of observed labels = -29701.93116875
iteration   3: log likelihood of observed labels = -30134.82557518
iteration   4: log likelihood of observed labels = -30722.29692008
iteration   5: log likelihood of observed labels = -31456.05698437
iteration   6: log likelihood of observed labels = -32328.49336598
iteration   7: log likelihood of observed labels = -33332.53999053
iteration   8: log likelihood of observed labels = -34461.59241521
iteration   9: log likelihood of observed labels = -35709.45022668
iteration  10: log likelihood of observed labels = -37070.27585371
iteration  11: log likelihood of observed labels = -38538.56337355
iteration  12: log likelihood of observed labels = -40109.11348365
iteration  13: log likelihood of observed labels = -41777.01238028
iteration  14: log likelihood of observed labels = -43537.6132

In [17]:
index = [str(i) for i in range(1, len(coefficients_1e5_penalty)+1)]
coefficients_1e5_penalty_df = pd.DataFrame(coefficients_1e5_penalty, index=index)

In [21]:
index = [str(i) for i in range(1, len(coefficients_1e3_penalty)+1)]
coefficients_1e3_penalty_df = pd.DataFrame(coefficients_1e3_penalty, index=index)
index = [str(i) for i in range(1, len(coefficients_1e2_penalty)+1)]
coefficients_1e2_penalty_df = pd.DataFrame(coefficients_1e2_penalty, index=index)
index = [str(i) for i in range(1, len(coefficients_10_penalty)+1)]
coefficients_10_penalty_df = pd.DataFrame(coefficients_10_penalty, index=index)
index = [str(i) for i in range(1, len(coefficients_4_penalty)+1)]
coefficients_4_penalty_df = pd.DataFrame(coefficients_4_penalty, index=index)
index = [str(i) for i in range(1, len(coefficients_0_penalty)+1)]
coefficients_0_penalty_df = pd.DataFrame(coefficients_0_penalty, index=index)

In [22]:
coefficients_0_penalty_df = pd.concat([coefficients_0_penalty_df, coefficients_4_penalty_df,coefficients_10_penalty_df,coefficients_1e2_penalty_df,coefficients_1e3_penalty_df,coefficients_1e5_penalty_df], axis=1)

In [31]:
coefficients_4_penalty

array([-0.0777747 ,  0.07440832,  0.01138698,  0.79318383,  1.03485777,
        0.00456272, -0.30465027, -0.01789266,  0.9793676 ,  0.54279474,
       -0.09943876,  0.19430276,  0.46459339, -0.17994936,  0.16125216,
        0.00388761,  0.15072199, -0.0722893 , -0.12615965, -0.26264138,
        0.16869205,  0.27904503, -0.00369078,  1.05179425, -0.03365981,
       -0.0029959 , -0.07043067,  0.19484338,  0.19774942, -0.26457855,
        0.07938766,  0.40986771, -0.0821891 , -0.388606  ,  0.85729344,
        0.44552773,  0.00320934, -0.09710669,  0.04493217, -0.09846741,
       -0.15442385,  0.09238314,  0.07004575,  0.03925715, -0.16706904,
        0.27884193,  0.04062695, -0.16973209,  0.34527228,  0.07200049,
       -0.28326893,  0.2773654 ,  0.32105061, -0.13841674, -0.06083573,
        0.17122948,  0.18688569, -0.16858369,  0.22570894,  0.08131243,
        0.02827011, -0.03212197,  0.15027508,  0.12103769,  0.08587272,
        0.10440364,  0.11550759, -0.30030678, -0.06822406, -0.20